In [1]:
from keras.models import Model, model_from_json, load_model
from keras.layers import Input, LSTM, Dense
#from keras.callbacks import ModelCheckpoint
import numpy as np
import keras
from tqdm import tqdm
import pandas as pd
#from keras.utils.vis_utils import plot_model

c:\python3.5\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
keras.__version__

'2.1.5'

### using pretrained model from module

In [3]:
from model import predict_form,predict_paradigm

In [ ]:
dec = load_model('decoder_full')

In [4]:
predict_form('огуречик23')

'огуречикам'

In [9]:
predict_paradigm('инвентарь')

,Singular,Plural
Nom,нивнетарь,инвентари
Gen,инвентаря,инвентарей
Dat,инвентарю,инвентарям
Acc,инвентарь,инвентари
Ins,инвентарём,инвентарями
Loc,инвентаре,инвентарях


### or step-by-step usage

In [6]:
#lines = open('C:/hse_compling/gits/ML-AI-experiments/AI/Neural Machine Translation/fra.txt', encoding='utf-8').read().split('\n')
lines = open('C:/hse_compling/diploma/data/data_lower_long.csv', encoding='utf-8').read().split('\n')
lines = lines[1:-1]

In [7]:
eng_sentences = []
fra_sentences = []
eng_chars = set()
fra_chars = set()
nb_samples = len(lines)

In [8]:
len(lines)

560640

In [9]:
# Process english and french sentences
for line in tqdm(range(nb_samples)):
    try:
        lemma,gender,formtag,classtag,form = lines[line].split('\t')
        eng_line = lemma+formtag
        #print(eng_line)
        # Append '\t' for start of the sentence and '\n' to signify end of the sentence
        fra_line = '\t' + form + '\n'
        #print(fra_line)
        eng_sentences.append(eng_line)
        fra_sentences.append(fra_line)

        for ch in eng_line:
            if (ch not in eng_chars):
                eng_chars.add(ch)

        for ch in fra_line:
            if (ch not in fra_chars):
                fra_chars.add(ch)
    except Exception as e:
        print(line, lines[line], e)

100%|████████████████████████████████████████| 560640/560640 [00:04<00:00, 130387.79it/s]


In [10]:
fra_chars = sorted(list(fra_chars))
eng_chars = sorted(list(eng_chars))

In [11]:
# dictionary to index each english character - key is index and value is english character
eng_index_to_char_dict = {}

# dictionary to get english character given its index - key is english character and value is index
eng_char_to_index_dict = {}

for k, v in enumerate(eng_chars):
    eng_index_to_char_dict[k] = v
    eng_char_to_index_dict[v] = k

In [18]:
# dictionary to index each french character - key is index and value is french character
fra_index_to_char_dict = {}

# dictionary to get french character given its index - key is french character and value is index
fra_char_to_index_dict = {}
for k, v in enumerate(fra_chars):
    fra_index_to_char_dict[k] = v
    fra_char_to_index_dict[v] = k

In [19]:
max_len_eng_sent = max([len(line) for line in eng_sentences])
max_len_fra_sent = max([len(line) for line in fra_sentences])

In [20]:
max_len_eng_sent

26

In [12]:
max_len_fra_sent

28

In [13]:
tokenized_eng_sentences = np.zeros(shape = (nb_samples,max_len_eng_sent,len(eng_chars)), dtype='float32')
tokenized_fra_sentences = np.zeros(shape = (nb_samples,max_len_fra_sent,len(fra_chars)), dtype='float32')
target_data = np.zeros((nb_samples, max_len_fra_sent, len(fra_chars)),dtype='float32')

In [14]:
tokenized_fra_sentences.shape

(560892, 28, 37)

In [13]:
encoder_model_inf = load_model('encoder.h5')
decoder_model_inf = load_model('decoder.h5')


c:\python3.5\lib\site-packages\keras\models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [35]:
def decode_seq(inp_seq):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
    
    target_seq = np.zeros((1, 1, len(fra_chars)))
    target_seq[0, 0, fra_char_to_index_dict['\t']] = 1
    
    translated_sent = ''
    stop_condition = False
    
    prob = 1.0
    while not stop_condition:
        
        #decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        #predict_pr = decoder_model_inf.predict_proba(x=[target_seq] + states_val)
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        #print(decoder_out)
        max_val_index = np.argmax(decoder_out[0,-1,:])
        max_val = np.max(decoder_out[0,-1,:])
        prob *= max_val
        sampled_fra_char = fra_index_to_char_dict[max_val_index]
        translated_sent += sampled_fra_char
        
        if ( (sampled_fra_char == '\n') or (len(translated_sent) > max_len_fra_sent)) :
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(fra_chars)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]
    
    prob /= len(translated_sent)
    return translated_sent, prob



In [30]:
def tokenize(seq):
    tokenized_eng_sentence = np.zeros(shape = (1,max_len_eng_sent,len(eng_chars)), dtype='float32')
    for k,ch in enumerate(seq):
        tokenized_eng_sentence[0,k,eng_char_to_index_dict[ch]] = 1
        
    return tokenized_eng_sentence


In [36]:
word = 'огуречик'
d = pd.DataFrame(np.zeros((6, 2)),index=['Nom','Gen','Dat','Acc','Ins','Loc',],columns=['Singular','Plural'])
tags = [[n,c,str(n)+str(c)] for n in [1,2] for c in [1,2,3,4,5,6]]
tag = '12'
inp = word + tag
inp_seq = tokenize(inp)
translated_sent, prob = decode_seq(inp_seq)
print(translated_sent, prob)

огуречика
 0.09996409422165156


In [22]:
word = 'огуречик'
d = pd.DataFrame(np.zeros((6, 2)),index=['Nom','Gen','Dat','Acc','Ins','Loc',],columns=['Singular','Plural'])
tags = [[n,c,str(n)+str(c)] for n in [1,2] for c in [1,2,3,4,5,6]]
for n,c,tag in tags:
    inp = word + tag
    inp_seq = tokenize(inp)
    translated_sent = decode_seq(inp_seq)
    d.iloc[c-1,n-1] = translated_sent.rstrip()
d

,Singular,Plural
Nom,огуречик,огуречики
Gen,огуречика,огуречиков
Dat,огуречику,огуречикам
Acc,огуречик,огуречиков
Ins,огуречиком,огуречиками
Loc,огуречике,огуречиках
